In [ ]:
%serialconnect --port COM5

serial exception on close WriteFile failed (PermissionError(13, '裝置無法辨識命令。', None, 22))
Connecting to --port=COM5 --baud=115200 
Ready.


In [ ]:
%sendtofile ap.txt
CHT-10-5
0953313123

Sent 2 lines (19 bytes) to ap.txt.


In [ ]:
%rebootdevice

normal repl mode not detected b'\r\nMicroPython v1.10-8-g8b7039d7d on 2019-01-26; ESP module with ESP8266\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\nPYB: soft rebTraceback (most recent call last):\r\n  File "boot.py", line 24, in <module>\r\nKeyboardInterrupt: \r\n'
not command mode
[Timed out waiting for recognizable response]


In [ ]:
import os
print(os.listdir())

['ap.txt', 'main.py', 'webtool.html', 'gy80.py', 'capture.py', 'quaternions.py', 'telescope_server.py']


In [ ]:
f = open('boot.py')
print(f.read())

# This file is executed on every boot (including wake-boot from deepsleep)
#import esp
#esp.osdebug(None)
import uos, machine
#uos.dupterm(None, 1) # disable REPL on UART(0)
import gc
import webrepl
webrepl.start()
gc.collect()


In [ ]:
import network
sta = network.WLAN(network.STA_IF)
sta.active(True)
sta.connect('CHT-10-5','0953313123')
sta.active()
print(sta.ifconfig())

('192.168.1.106', '255.255.255.0', '192.168.1.1', '192.168.1.1')


In [ ]:
%sendtofile boot.py
try:
  import usocket as socket
except:
  import socket

import network
from machine import Pin
import dht

import esp
esp.osdebug(None)

import gc
gc.collect()

ssid = 'CHT-10-5'
password = '0953313123'

station = network.WLAN(network.STA_IF)

station.active(True)
station.connect(ssid, password)

while station.isconnected() == False:
  pass

print('Connection successful')
print(station.ifconfig())

#sensor = dht.DHT22(Pin(14))
sensor = dht.DHT11(Pin(5))

Sent 31 lines (458 bytes) to boot.py.


In [ ]:
%sendtofile main.py
# Complete project details at https://RandomNerdTutorials.com

def read_sensor():
  global temp, hum
  temp = hum = 0
  try:
    sensor.measure()
    temp = sensor.temperature()
    hum = sensor.humidity()
    if (isinstance(temp, float) and isinstance(hum, float)) or (isinstance(temp, int) and isinstance(hum, int)):
      msg = (b'{0:3.1f},{1:3.1f}'.format(temp, hum))

      # uncomment for Fahrenheit
      #temp = temp * (9/5) + 32.0

      hum = round(hum, 2)
      return(msg)
    else:
      return('Invalid sensor readings.')
  except OSError as e:
    return('Failed to read sensor.')

def web_page():
  html = """<!DOCTYPE HTML><html>
<head>
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <link rel="stylesheet" href="https://use.fontawesome.com/releases/v5.7.2/css/all.css" integrity="sha384-fnmOCqbTlWIlj8LyTjo7mOUStjsKC4pOpQbqyi7RrhN7udi9RwhKkMHpvLbHG9Sr" crossorigin="anonymous">
  <style>
    html {
     font-family: Arial;
     display: inline-block;
     margin: 0px auto;
     text-align: center;
    }
    h2 { font-size: 3.0rem; }
    p { font-size: 3.0rem; }
    .units { font-size: 1.2rem; }
    .dht-labels{
      font-size: 1.5rem;
      vertical-align:middle;
      padding-bottom: 15px;
    }
  </style>
</head>
<body>
  <h2>ESP32 DHT Server</h2>
  <p>
    <i class="fas fa-thermometer-half" style="color:#059e8a;"></i> 
    <span class="dht-labels">Temperature</span> 
    <span>"""+str(temp)+"""</span>
    <sup class="units">&deg;C</sup>
  </p>
  <p>
    <i class="fas fa-tint" style="color:#00add6;"></i> 
    <span class="dht-labels">Humidity</span>
    <span>"""+str(hum)+"""</span>
    <sup class="units">%</sup>
  </p>
</body>
</html>"""
  return html

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind(('', 80))
s.listen(5)

while True:
  conn, addr = s.accept()
  print('Got a connection from %s' % str(addr))
  request = conn.recv(1024)
  print('Content = %s' % str(request))
  sensor_readings = read_sensor()
  print(sensor_readings)
  response = web_page()
  conn.send('HTTP/1.1 200 OK\n')
  conn.send('Content-Type: text/html\n')
  conn.send('Connection: close\n\n')
  conn.sendall(response)
  conn.close()

Sent 79 lines (2186 bytes) to main.py.


In [ ]:
'''
<i class="fas fa-thermometer-half" style="color:#00add6;"></i>
<i class="fas fa-humidity" style="color:#00add6;"></i>
<p>
  <i class="fas fa-tint" style="color:#00add6;"></i> 
  <span class="dht-labels">Humidity</span>
  <span>"""+str(hum)+"""</span>
  <sup class="units">%</sup>
</p>
'''

In [ ]:
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind(('', 80))
s.listen(5)

while True:
  conn, addr = s.accept()
  print('Got a connection from %s' % str(addr))
  request = conn.recv(1024)
  print('Content = %s' % str(request))
  sensor_readings = read_sensor()
  print(sensor_readings)
  response = web_page()
  conn.send('HTTP/1.1 200 OK\n')
  conn.send('Content-Type: text/html\n')
  conn.send('Connection: close\n\n')
  conn.sendall(response)
  conn.close()